# Spotify Exploration

As of now (8/17/2023) I have only done this in R. So before we create the functions, we will recreate the functionality of my R script. 

1. Install Necessary Packages
2. Set system environments appropriately for Spotify ID's
3. Experiment with the Spotipy package
a. Get songs from an artist
b. Using track_id, get the song info (length, tempo, danceability, energy)
4. How many songs do I have saved?
5. Read in names of all songs
6. Get relevant data from all songs
7. Manage the dataframe
8. Create playlist groupings by tempo
9. Create playlist groupings by danceability
10. Create playlist groupings by energy
11. Write Playlist
12. Work with functions

# 1. Import Packages

In [2]:
import config
import pandas as pd
import numpy as np
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import math
import time

# 2. Authorization

Here we'll keep an example code from Spotipy for client authorizaiton (search spotify database but not any user data)
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

playlists = sp.user_playlists('spotify')
while playlists:
    for i, playlist in enumerate(playlists['items']):
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

Set system environments

In [3]:
os.environ["SPOTIPY_CLIENT_ID"] = config.SPOTIPY_CLIENT_ID
os.environ["SPOTIPY_CLIENT_SECRET"] = config.SPOTIPY_CLIENT_SECRET
os.environ["SPOTIPY_REDIRECT_URI"] = config.SPOTIPY_REDIRECT_URI

Set scope & do a data pull

* Note that "user-library-read" is the only necessary scope to read saved tracks

In [4]:
scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

profile = sp.current_user()
results = sp.current_user_saved_tracks(limit = 20, offset = 0, market = None)

# Item is a specific "row". Or a single song
for idx, item in enumerate(results['items']):
    
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])

0 Lil Uzi Vert  –  Watch This - ARIZONATEARS Pluggnb Remix
1 Post Malone  –  Don't Understand
2 Dominic Fike  –  3 Nights
3 Dominic Fike  –  Double Negative (Skeleton Milkshake)
4 Post Malone  –  Something Real
5 Lit  –  My Own Worst Enemy
6 J. Cole  –  h u n g e r . o n . h i l l s i d e (with Bas)
7 J. Cole  –  a p p l y i n g . p r e s s u r e
8 J. Cole  –  m y . l i f e (with 21 Savage & Morray)
9 J. Cole  –  p u n c h i n ‘ . t h e . c l o c k
10 J. Cole  –  9 5 . s o u t h
11 J. Cole  –  a m a r i
12 BlocBoy JB  –  Look Alive (feat. Drake)
13 J. Cole  –  1 0 0 . m i l ‘ (with Bas)
14 jxdn  –  ANGELS & DEMONS
15 The White Stripes  –  Seven Nation Army
16 Nirvana  –  Come As You Are
17 Sublime  –  Santeria
18 100 gecs  –  Hollywood Baby
19 Lovejoy  –  Call Me What You Like


For each song that you get from current_user_saved_tracks, the variable will have two pieces ('added at', and 'track')

In [8]:
results['items'][0]['track']['id']

'0FA4wrjDJvJTTU8AepZTup'

In [5]:
print('you have', results['total'], 'saved songs')

you have 4412 saved songs


In [6]:
profile

{'display_name': 'Dante Goss',
 'external_urls': {'spotify': 'https://open.spotify.com/user/1218158724'},
 'href': 'https://api.spotify.com/v1/users/1218158724',
 'id': '1218158724',
 'images': [{'url': 'https://scontent-sea1-1.xx.fbcdn.net/v/t39.30808-1/287739885_5170783559641650_7329131775805179735_n.jpg?stp=cp0_dst-jpg_p50x50&_nc_cat=108&ccb=1-7&_nc_sid=dbb9e7&_nc_ohc=k_WMIxgYS_gAX_9-18Z&_nc_ht=scontent-sea1-1.xx&edm=AP4hL3IEAAAA&oh=00_AfB4chcvxYSWVwW_RFoGMcsuq0Kr59kgMHX3OLWHHW9N_g&oe=64E583B5',
   'height': 64,
   'width': 64},
  {'url': 'https://scontent-sea1-1.xx.fbcdn.net/v/t39.30808-1/287739885_5170783559641650_7329131775805179735_n.jpg?stp=dst-jpg_p320x320&_nc_cat=108&ccb=1-7&_nc_sid=0c64ff&_nc_ohc=k_WMIxgYS_gAX_9-18Z&_nc_ht=scontent-sea1-1.xx&edm=AP4hL3IEAAAA&oh=00_AfBoB0rWAKun62cOmvLti53YbXDdTInKzdxcL9HUTjV2qw&oe=64E583B5',
   'height': 300,
   'width': 300}],
 'type': 'user',
 'uri': 'spotify:user:1218158724',
 'followers': {'href': None, 'total': 15},
 'country': 'US',
 'p

In [7]:
# item is a dictionary
print('track id: ', item['track']['id'])

print('track name: ', item['track']['name'])

print('track length in seconds: ',item['track']['duration_ms']/1000)

print('album name: ',item['track']['album']['name'])

print('first artist: ', item['track']['artists'][0]['name'])

print('number of artists: ', len(item['track']['artists']))


track id:  2QF8FbGBTXTzm0CRUWqndE
track name:  Call Me What You Like
track length in seconds:  226.96
album name:  Wake Up & It's Over
first artist:  Lovejoy
number of artists:  1


# First loop
This loop is meant to go through all saved songs and get name, id, artist name, artist id, number of artists, and track length in seconds.

In [20]:
# Now we need to make a loop that will go through all saved songs
# Set scope
scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"
# Authorize
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
# Step 1: How many saved songs do we have?
# You can get this by doing a current_user_saved_tracks search
song = sp.current_user_saved_tracks(limit = 1, offset = 0, market = None)
num_songs = song['total']

# Initialize Variables
track_name = [] 
track_id = []
artist_name = []
artist_id = []
artist_num = []
track_len = []
# The limit for current_user_saved_tracks is 20, so we need to round up to the nearest multiple of 20
# in the math package the ceil function will round up
num_loops = math.ceil(song['total']/20)

# Now make a big loop that will go through it all
for i in range(0,num_loops+1):
    print('Loop Iteration',i+1)
    result = sp.current_user_saved_tracks(limit = 20, offset = (i*20), market = None)
    # sleep
    time.sleep(3)# 3 second sleep
    # Loop through saved tracks
    for item in result['items']:
        track = item['track']
        track_name.append(track['name'])
        track_id.append(track['id'])   
        artist_name.append(track['artists'][0]['name'])
        artist_id.append(track['artists'][0]['id'])
        artist_num.append(len(track['artists']))
        track_len.append(track['duration_ms']/1000)
# Convert to DF
Track_Name=pd.DataFrame(track_name,columns=['Track_Name'])
Track_ID=pd.DataFrame(track_id,columns=['Track_ID'])
Artist_Name=pd.DataFrame(artist_name,columns=['Artist_Name'])
Artist_ID=pd.DataFrame(artist_id,columns=['Artist_ID'])
Artist_Num=pd.DataFrame(artist_num,columns=['Artist_Num'])
Track_Len=pd.DataFrame(track_len,columns=['Track_Len'])
# Combine
df = pd.concat([Track_Name,Track_ID,Artist_Name,Artist_ID,Artist_Num,Track_Len],axis =1)
df



Loop Iteration 1
Loop Iteration 2
Loop Iteration 3
Loop Iteration 4
Loop Iteration 5
Loop Iteration 6
Loop Iteration 7
Loop Iteration 8
Loop Iteration 9
Loop Iteration 10
Loop Iteration 11
Loop Iteration 12
Loop Iteration 13
Loop Iteration 14
Loop Iteration 15
Loop Iteration 16
Loop Iteration 17
Loop Iteration 18
Loop Iteration 19
Loop Iteration 20
Loop Iteration 21
Loop Iteration 22
Loop Iteration 23
Loop Iteration 24
Loop Iteration 25
Loop Iteration 26
Loop Iteration 27
Loop Iteration 28
Loop Iteration 29
Loop Iteration 30
Loop Iteration 31
Loop Iteration 32
Loop Iteration 33
Loop Iteration 34
Loop Iteration 35
Loop Iteration 36
Loop Iteration 37
Loop Iteration 38
Loop Iteration 39
Loop Iteration 40
Loop Iteration 41
Loop Iteration 42
Loop Iteration 43
Loop Iteration 44
Loop Iteration 45
Loop Iteration 46
Loop Iteration 47
Loop Iteration 48
Loop Iteration 49
Loop Iteration 50
Loop Iteration 51
Loop Iteration 52
Loop Iteration 53
Loop Iteration 54
Loop Iteration 55
Loop Iteration 56
L

,Track_Name,Track_ID,Artist_Name,Artist_ID,Artist_Num,Track_Len
0,Watch This - ARIZONATEARS Pluggnb Remix,0FA4wrjDJvJTTU8AepZTup,Lil Uzi Vert,4O15NlyKLIASxsJ0PrXPfz,3,163.139
1,Don't Understand,4MTuL20LF3pWebeJbcNh7p,Post Malone,246dkjvS1zLTtiykXe5h60,1,183.379
2,3 Nights,0uI7yAKUf52Cn7y3sYyjiX,Dominic Fike,6USv9qhCn6zfxlBQIYJ9qs,1,177.666
3,Double Negative (Skeleton Milkshake),7ACT6YaXbYvl7hRWEOOEHQ,Dominic Fike,6USv9qhCn6zfxlBQIYJ9qs,1,126.465
4,Something Real,444vevlQjTnKioLLncteGv,Post Malone,246dkjvS1zLTtiykXe5h60,1,205.287
...,...,...,...,...,...,...
4407,Started From the Bottom,6V2D8Lls36APk0THDjBDfE,Drake,3TVXtAsR1Inumwj472S9r4,1,174.133
4408,The Language,3Qu5bTS5AvgS0TpeGhQyfc,Drake,3TVXtAsR1Inumwj472S9r4,1,224.013
4409,All Me,4kNvYhyl8R6m1vykVkcuBu,Drake,3TVXtAsR1Inumwj472S9r4,3,270.866
4410,Forever,6HSqyfGnsHYw9MmIpa9zlZ,Drake,3TVXtAsR1Inumwj472S9r4,4,357.346


In [23]:
# Now we can do a loop that gets key info for each track
# This version will be slow because it's one at a time.
# Set scope
scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"
# Authorize
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

#trackfeatures = []
#for tracks in track_id:
    #trackfeatures.append( sp.audio_features(tracks = tracks) )
    #time.sleep(3)
#track_features = pd.DataFrame(trackfeatures)

# Step 1: How many saved songs do we have?
# You can get this by doing a current_user_saved_tracks search
song = sp.current_user_saved_tracks(limit = 1, offset = 0, market = None)
num_songs = song['total']
num_loops = math.ceil(song['total']/20)

# Realistically, we can do up to 100 searches at a time from the spotipy package
scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"
# Authorize
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

trackfeatures = pd.DataFrame(columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'])

for i in range(0,num_loops-1):
#for i in range(0,30):
    firstnum = i*20
    secondnum = firstnum +20
    #print(firstnum,secondnum)
    audio = (sp.audio_features(tracks = track_id[firstnum:secondnum]))
    audio = pd.DataFrame(audio)
    audio = audio.reset_index(inplace=False)
    
    trackfeatures = pd.concat([trackfeatures,audio],axis=0,ignore_index=True)
    
    time.sleep(2.1)
    idxval = num_loops-1-i
    print(idxval,'Loops to go! :)')
    if idxval <1:
        print("\a")# alarm?
        print('We have liftoff!')
        print('Come back to your code!')


# Now Merge with df
trackfeatures =trackfeatures.rename(columns={'id':'Track_ID'})
data = pd.merge(df,trackfeatures,on=['Track_ID'])
data

222 Loops to go! :)
221 Loops to go! :)
220 Loops to go! :)
219 Loops to go! :)
218 Loops to go! :)
217 Loops to go! :)
216 Loops to go! :)
215 Loops to go! :)
214 Loops to go! :)
213 Loops to go! :)
212 Loops to go! :)
211 Loops to go! :)
210 Loops to go! :)
209 Loops to go! :)
208 Loops to go! :)
207 Loops to go! :)
206 Loops to go! :)
205 Loops to go! :)
204 Loops to go! :)
203 Loops to go! :)
202 Loops to go! :)
201 Loops to go! :)
200 Loops to go! :)
199 Loops to go! :)
198 Loops to go! :)
197 Loops to go! :)
196 Loops to go! :)
195 Loops to go! :)
194 Loops to go! :)
193 Loops to go! :)
192 Loops to go! :)
191 Loops to go! :)
190 Loops to go! :)
189 Loops to go! :)
188 Loops to go! :)
187 Loops to go! :)
186 Loops to go! :)
185 Loops to go! :)
184 Loops to go! :)
183 Loops to go! :)
182 Loops to go! :)
181 Loops to go! :)
180 Loops to go! :)
179 Loops to go! :)
178 Loops to go! :)
177 Loops to go! :)
176 Loops to go! :)
175 Loops to go! :)
174 Loops to go! :)
173 Loops to go! :)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,index
0,0.686,0.897,11,-7.180,0,0.0386,0.010300,0.103000,0.153,0.355,129.975,audio_features,0FA4wrjDJvJTTU8AepZTup,spotify:track:0FA4wrjDJvJTTU8AepZTup,https://api.spotify.com/v1/tracks/0FA4wrjDJvJT...,https://api.spotify.com/v1/audio-analysis/0FA4...,163139,4,0.0
1,0.413,0.316,7,-7.794,1,0.0439,0.586000,0.000003,0.105,0.319,183.328,audio_features,4MTuL20LF3pWebeJbcNh7p,spotify:track:4MTuL20LF3pWebeJbcNh7p,https://api.spotify.com/v1/tracks/4MTuL20LF3pW...,https://api.spotify.com/v1/audio-analysis/4MTu...,183379,4,1.0
2,0.815,0.518,7,-6.594,0,0.0897,0.223000,0.000000,0.104,0.877,151.891,audio_features,0uI7yAKUf52Cn7y3sYyjiX,spotify:track:0uI7yAKUf52Cn7y3sYyjiX,https://api.spotify.com/v1/tracks/0uI7yAKUf52C...,https://api.spotify.com/v1/audio-analysis/0uI7...,177667,4,2.0
3,0.689,0.858,4,-2.868,1,0.1610,0.003830,0.000000,0.129,0.643,153.977,audio_features,7ACT6YaXbYvl7hRWEOOEHQ,spotify:track:7ACT6YaXbYvl7hRWEOOEHQ,https://api.spotify.com/v1/tracks/7ACT6YaXbYvl...,https://api.spotify.com/v1/audio-analysis/7ACT...,126465,4,3.0
4,0.445,0.797,11,-5.086,0,0.0434,0.000227,0.000000,0.363,0.267,135.483,audio_features,444vevlQjTnKioLLncteGv,spotify:track:444vevlQjTnKioLLncteGv,https://api.spotify.com/v1/tracks/444vevlQjTnK...,https://api.spotify.com/v1/audio-analysis/444v...,205288,4,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.628,0.601,1,-7.218,1,0.0336,0.745000,0.000000,0.310,0.744,114.988,audio_features,6g7SBkXVYCnnVVv8NE3lfV,spotify:track:6g7SBkXVYCnnVVv8NE3lfV,https://api.spotify.com/v1/tracks/6g7SBkXVYCnn...,https://api.spotify.com/v1/audio-analysis/6g7S...,192862,4,15.0
4396,0.508,0.763,1,-5.674,1,0.3030,0.062700,0.000000,0.347,0.275,161.590,audio_features,4zGvb8hxGLB2jEPRFiRRqw,spotify:track:4zGvb8hxGLB2jEPRFiRRqw,https://api.spotify.com/v1/tracks/4zGvb8hxGLB2...,https://api.spotify.com/v1/audio-analysis/4zGv...,192947,4,16.0
4397,0.430,0.955,9,-4.767,1,0.0424,0.000807,0.000000,0.044,0.366,172.975,audio_features,6HNruFQlzQx4ulL4ppRLYI,spotify:track:6HNruFQlzQx4ulL4ppRLYI,https://api.spotify.com/v1/tracks/6HNruFQlzQx4...,https://api.spotify.com/v1/audio-analysis/6HNr...,267360,4,17.0
4398,0.695,0.628,5,-8.627,1,0.1450,0.337000,0.219000,0.143,0.210,119.949,audio_features,4oRqSiVyGrwb8dnJBO6FNu,spotify:track:4oRqSiVyGrwb8dnJBO6FNu,https://api.spotify.com/v1/tracks/4oRqSiVyGrwb...,https://api.spotify.com/v1/audio-analysis/4oRq...,244653,4,18.0


In [32]:
def NumSavedSongs():
    # Set scope
    scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"
    # Authorize
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
    song = sp.current_user_saved_tracks(limit = 1, offset = 0, market = None)
    num_songs = song['total']
    return num_songs

Combine both loops

In [9]:
# Now we need to make a loop that will go through all saved songs
# Set scope
scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"
# Authorize
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
# Step 1: How many saved songs do we have?
# You can get this by doing a current_user_saved_tracks search
song = sp.current_user_saved_tracks(limit = 1, offset = 0, market = None)
num_songs = song['total']

# Initialize Variables
track_name = [] 
track_id = []
artist_name = []
artist_id = []
artist_num = []
track_len = []
trackfeatures = pd.DataFrame(columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'])
# The limit for current_user_saved_tracks is 20, so we need to round up to the nearest multiple of 20
# in the math package the ceil function will round up
num_loops = math.ceil(song['total']/20)

# Now make a big loop that will go through it all
for i in range(0,num_loops+1):
    print((num_loops+1-i),'Loops Left')
    result = sp.current_user_saved_tracks(limit = 20, offset = (i*20), market = None)
    # sleep
    time.sleep(5)# 3 second sleep
    # Loop through saved tracks
    for item in result['items']:
        track = item['track']
        track_name.append(track['name'])
        track_id.append(track['id'])  
        audio = (sp.audio_features(tracks = track['id']))
        audio = pd.DataFrame(audio)
        audio = audio.reset_index(inplace=False)
        trackfeatures = pd.concat([trackfeatures,audio],axis=0,ignore_index=True)
         
        artist_name.append(track['artists'][0]['name'])
        artist_id.append(track['artists'][0]['id'])
        artist_num.append(len(track['artists']))
        track_len.append(track['duration_ms']/1000)
# Convert to DF
Track_Name=pd.DataFrame(track_name,columns=['Track_Name'])
Track_ID=pd.DataFrame(track_id,columns=['Track_ID'])
Artist_Name=pd.DataFrame(artist_name,columns=['Artist_Name'])
Artist_ID=pd.DataFrame(artist_id,columns=['Artist_ID'])
Artist_Num=pd.DataFrame(artist_num,columns=['Artist_Num'])
Track_Len=pd.DataFrame(track_len,columns=['Track_Len'])
# Combine
df = pd.concat([Track_Name,Track_ID,Artist_Name,Artist_ID,Artist_Num,Track_Len],axis =1)
df

# Now Merge with df
trackfeatures =trackfeatures.rename(columns={'id':'Track_ID'})
data = pd.merge(df,trackfeatures,on=['Track_ID'])
data

# took 28 min

221 Loops Left
220 Loops Left
219 Loops Left
218 Loops Left
217 Loops Left
216 Loops Left
215 Loops Left
214 Loops Left
213 Loops Left
212 Loops Left
211 Loops Left
210 Loops Left
209 Loops Left
208 Loops Left
207 Loops Left
206 Loops Left
205 Loops Left
204 Loops Left
203 Loops Left
202 Loops Left
201 Loops Left
200 Loops Left
199 Loops Left
198 Loops Left
197 Loops Left
196 Loops Left
195 Loops Left
194 Loops Left
193 Loops Left
192 Loops Left
191 Loops Left
190 Loops Left
189 Loops Left
188 Loops Left
187 Loops Left
186 Loops Left
185 Loops Left
184 Loops Left
183 Loops Left
182 Loops Left
181 Loops Left
180 Loops Left
179 Loops Left
178 Loops Left
177 Loops Left
176 Loops Left
175 Loops Left
174 Loops Left
173 Loops Left
172 Loops Left
171 Loops Left
170 Loops Left
169 Loops Left
168 Loops Left
167 Loops Left
166 Loops Left
165 Loops Left
164 Loops Left
163 Loops Left
162 Loops Left
161 Loops Left
160 Loops Left
159 Loops Left
158 Loops Left
157 Loops Left
156 Loops Left
155 Loops 

,Track_Name,Track_ID,Artist_Name,Artist_ID,Artist_Num,Track_Len,danceability,energy,key,loudness,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,index
0,Watch This - ARIZONATEARS Pluggnb Remix,0FA4wrjDJvJTTU8AepZTup,Lil Uzi Vert,4O15NlyKLIASxsJ0PrXPfz,3,163.139,0.686,0.897,11,-7.180,...,0.1530,0.355,129.975,audio_features,spotify:track:0FA4wrjDJvJTTU8AepZTup,https://api.spotify.com/v1/tracks/0FA4wrjDJvJT...,https://api.spotify.com/v1/audio-analysis/0FA4...,163139,4,0.0
1,Don't Understand,4MTuL20LF3pWebeJbcNh7p,Post Malone,246dkjvS1zLTtiykXe5h60,1,183.379,0.413,0.316,7,-7.794,...,0.1050,0.319,183.328,audio_features,spotify:track:4MTuL20LF3pWebeJbcNh7p,https://api.spotify.com/v1/tracks/4MTuL20LF3pW...,https://api.spotify.com/v1/audio-analysis/4MTu...,183379,4,0.0
2,3 Nights,0uI7yAKUf52Cn7y3sYyjiX,Dominic Fike,6USv9qhCn6zfxlBQIYJ9qs,1,177.666,0.815,0.518,7,-6.594,...,0.1040,0.877,151.891,audio_features,spotify:track:0uI7yAKUf52Cn7y3sYyjiX,https://api.spotify.com/v1/tracks/0uI7yAKUf52C...,https://api.spotify.com/v1/audio-analysis/0uI7...,177667,4,0.0
3,Double Negative (Skeleton Milkshake),7ACT6YaXbYvl7hRWEOOEHQ,Dominic Fike,6USv9qhCn6zfxlBQIYJ9qs,1,126.465,0.689,0.858,4,-2.868,...,0.1290,0.643,153.977,audio_features,spotify:track:7ACT6YaXbYvl7hRWEOOEHQ,https://api.spotify.com/v1/tracks/7ACT6YaXbYvl...,https://api.spotify.com/v1/audio-analysis/7ACT...,126465,4,0.0
4,Something Real,444vevlQjTnKioLLncteGv,Post Malone,246dkjvS1zLTtiykXe5h60,1,205.287,0.445,0.797,11,-5.086,...,0.3630,0.267,135.483,audio_features,spotify:track:444vevlQjTnKioLLncteGv,https://api.spotify.com/v1/tracks/444vevlQjTnK...,https://api.spotify.com/v1/audio-analysis/444v...,205288,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4407,Started From the Bottom,6V2D8Lls36APk0THDjBDfE,Drake,3TVXtAsR1Inumwj472S9r4,1,174.133,0.794,0.522,8,-7.829,...,0.1560,0.567,86.318,audio_features,spotify:track:6V2D8Lls36APk0THDjBDfE,https://api.spotify.com/v1/tracks/6V2D8Lls36AP...,https://api.spotify.com/v1/audio-analysis/6V2D...,174133,4,0.0
4408,The Language,3Qu5bTS5AvgS0TpeGhQyfc,Drake,3TVXtAsR1Inumwj472S9r4,1,224.013,0.831,0.651,1,-8.363,...,0.0809,0.502,120.064,audio_features,spotify:track:3Qu5bTS5AvgS0TpeGhQyfc,https://api.spotify.com/v1/tracks/3Qu5bTS5AvgS...,https://api.spotify.com/v1/audio-analysis/3Qu5...,224013,4,0.0
4409,All Me,4kNvYhyl8R6m1vykVkcuBu,Drake,3TVXtAsR1Inumwj472S9r4,3,270.866,0.670,0.515,1,-6.977,...,0.2500,0.422,122.006,audio_features,spotify:track:4kNvYhyl8R6m1vykVkcuBu,https://api.spotify.com/v1/tracks/4kNvYhyl8R6m...,https://api.spotify.com/v1/audio-analysis/4kNv...,270867,4,0.0
4410,Forever,6HSqyfGnsHYw9MmIpa9zlZ,Drake,3TVXtAsR1Inumwj472S9r4,4,357.346,0.415,0.926,5,-1.304,...,0.2560,0.542,158.423,audio_features,spotify:track:6HSqyfGnsHYw9MmIpa9zlZ,https://api.spotify.com/v1/tracks/6HSqyfGnsHYw...,https://api.spotify.com/v1/audio-analysis/6HSq...,357347,4,0.0
